In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
OUTPUT_POSTPROCESSING_DIR_PATH = os.getcwd()
SSP_MODELING_DIR_PATH = os.path.dirname(OUTPUT_POSTPROCESSING_DIR_PATH)
SSP_OUTPUT_DIR_PATH = os.path.join(SSP_MODELING_DIR_PATH, "ssp_run_output")
CW_DATA_DIR_PATH = os.path.join(OUTPUT_POSTPROCESSING_DIR_PATH, "data")

In [3]:
ISO3 = "MEX"
REGION_NAME = "mexico"
RUN_DIR_PATH = os.path.join(SSP_OUTPUT_DIR_PATH, "sisepuede_results_sisepuede_run_2025-10-23T13;32;38.383522")

### Load emission targets and ssp outputs dfs

In [4]:
# Load emission targets
emission_targets_df = pd.read_csv(os.path.join(CW_DATA_DIR_PATH, "emission_targets_mexico_2022.csv"))
emission_targets_df.head()

,Subsector,Gas,Edgar_Class,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,ids,MEX
0,agrc,ch4,AG - Crops:CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,1:agrc:ch4,1.144799
1,agrc,co2,AG - Crops:CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,2:agrc:co2,2.938056
2,agrc,n2o,AG - Crops:N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,3:agrc:n2o,29.452258
3,lvst,ch4,AG - Livestock:CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,4:lvst:ch4,44.017738
4,lsmm,ch4,AG - Livestock:CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,5:lsmm:ch4,44.017738


In [5]:
# Load output data
ssp_output_df = pd.read_csv(os.path.join(RUN_DIR_PATH, 
                                         "sisepuede_results_sisepuede_run_2025-10-23T13;32;38.383522_WIDE_INPUTS_OUTPUTS.csv"))
ssp_output_df.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,mexico,0,938860.188587,1.295621e+07,392567.842328,1.910190e+06,19046.794337,271808.943979,2.848687e+06,...,0.0,0.886588,0.656536,0.0,0.684783,5.246665,0.0,5.171046,3.278876,92.81
1,0,mexico,1,942528.969886,1.300684e+07,394101.878573,1.917654e+06,19121.223441,272871.091019,2.859819e+06,...,0.0,0.879202,0.656977,0.0,0.695971,5.623655,0.0,5.215653,3.250657,92.81
2,0,mexico,2,944066.850803,1.302806e+07,394744.916376,1.920783e+06,19152.422657,273316.322155,2.864485e+06,...,0.0,0.882950,0.655764,0.0,0.697514,5.742704,0.0,5.238361,3.285214,92.81
3,0,mexico,3,942372.313186,1.300468e+07,394036.375335,1.917336e+06,19118.045323,272825.737416,2.859343e+06,...,0.0,0.887800,0.651245,0.0,0.686271,3.858085,0.0,5.091276,3.321646,92.81
4,0,mexico,4,935865.150162,1.291488e+07,391315.519792,1.904096e+06,18986.033553,270941.851901,2.839599e+06,...,0.0,0.882730,0.646505,0.0,0.696880,4.722076,0.0,5.101050,3.355597,92.81


### Obtain the ssp output values in the emission targets format

In [6]:
def sum_vars_from_ssp_outputs(
    emission_targets_df: pd.DataFrame,
    ssp_outputs_df: pd.DataFrame,
    vars_col: str = "Vars",
    out_col: str = "ssp_total",
    record_missing_col: str | None = "missing_vars",
    ssp_filter: dict | None = None,
) -> pd.DataFrame:
    """
    For each row in emission_targets_df, split the colon-separated strings in `vars_col`,
    find those columns in ssp_outputs_df, sum their values (over rows & columns), and
    write the total to `out_col` in emission_targets_df.

    Parameters
    ----------
    emission_targets_df : DataFrame
        Must contain a string column `vars_col` with colon-separated names.
    ssp_outputs_df : DataFrame
        Wide table whose columns include the names referenced by `emission_targets_df[vars_col]`.
    vars_col : str
        Column in emission_targets_df with colon-separated variable names.
    out_col : str
        New column to create in emission_targets_df with totals from ssp_outputs_df.
    record_missing_col : str | None
        If provided, creates a column listing any missing vars for each row.
    df2_filter : dict | None
        Optional filters to reduce ssp_outputs_df before summing, e.g.
        {"region": "egypt", "time_period": 7}

    Returns
    -------
    DataFrame
        emission_targets_df with new column `out_col` (and `record_missing_col` if requested).
    """
    # Optionally filter ssp_outputs_df by key=value pairs (e.g., region/time_period)
    if ssp_filter:
        mask = pd.Series(True, index=ssp_outputs_df.index)
        for k, v in ssp_filter.items():
            mask &= (ssp_outputs_df[k] == v)
        ssp_view = ssp_outputs_df.loc[mask]
    else:
        ssp_view = ssp_outputs_df

    # Ensure we only operate on numeric data when summing
    numeric_cols = set(ssp_view.select_dtypes(include=[np.number]).columns)

    def _total_for_vars(vars_str: str):
        if pd.isna(vars_str) or not str(vars_str).strip():
            return np.nan, []

        # Split, strip, and deduplicate while preserving order
        raw = [s.strip() for s in str(vars_str).split(":") if s.strip()]
        seen = set()
        cols = [c for c in raw if not (c in seen or seen.add(c))]

        present = [c for c in cols if c in ssp_view.columns and c in numeric_cols]
        missing = [c for c in cols if c not in ssp_view.columns or c not in numeric_cols]

        if not present or ssp_view.empty:
            return np.nan, missing

        # Sum over all filtered rows & all present columns
        vals = ssp_view[present].to_numpy(dtype=float, copy=False)
        total = np.nansum(vals)
        return float(total), missing

    totals, missings = [], []
    for v in emission_targets_df[vars_col].astype("string"):
        total, missing = _total_for_vars(v)
        totals.append(total)
        missings.append(missing)

    emission_targets_df = emission_targets_df.copy()
    emission_targets_df[out_col] = totals
    if record_missing_col is not None:
        emission_targets_df[record_missing_col] = missings

    return emission_targets_df


# -----------------------------
# Example usage
# -----------------------------

# If DF2 has a single row for the target (e.g., region="egypt", a specific time_period):
# df2_filter = {"region": "egypt"}              # or {"region": "egypt", "time_period": 7}
# If you want to sum across all rows of DF2, set df2_filter = None.

# df1_result = sum_vars_from_df2(DF1, DF2, vars_col="Vars",
#                                out_col="DF2_total",
#                                record_missing_col="Missing_in_DF2",
#                                df2_filter={"region": "egypt"})
# print(df1_result.head())


In [7]:
ssp_output_df.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,mexico,0,938860.188587,1.295621e+07,392567.842328,1.910190e+06,19046.794337,271808.943979,2.848687e+06,...,0.0,0.886588,0.656536,0.0,0.684783,5.246665,0.0,5.171046,3.278876,92.81
1,0,mexico,1,942528.969886,1.300684e+07,394101.878573,1.917654e+06,19121.223441,272871.091019,2.859819e+06,...,0.0,0.879202,0.656977,0.0,0.695971,5.623655,0.0,5.215653,3.250657,92.81
2,0,mexico,2,944066.850803,1.302806e+07,394744.916376,1.920783e+06,19152.422657,273316.322155,2.864485e+06,...,0.0,0.882950,0.655764,0.0,0.697514,5.742704,0.0,5.238361,3.285214,92.81
3,0,mexico,3,942372.313186,1.300468e+07,394036.375335,1.917336e+06,19118.045323,272825.737416,2.859343e+06,...,0.0,0.887800,0.651245,0.0,0.686271,3.858085,0.0,5.091276,3.321646,92.81
4,0,mexico,4,935865.150162,1.291488e+07,391315.519792,1.904096e+06,18986.033553,270941.851901,2.839599e+06,...,0.0,0.882730,0.646505,0.0,0.696880,4.722076,0.0,5.101050,3.355597,92.81


In [8]:
emission_targets_df_extended = sum_vars_from_ssp_outputs(emission_targets_df, ssp_output_df, vars_col="Vars",
                               out_col="ssp_emission",
                               record_missing_col="missing_in_ssp_outputs",
                               ssp_filter={"region": REGION_NAME, "primary_id": 0, "time_period": 7})

emission_targets_df_extended.head()

,Subsector,Gas,Edgar_Class,Edgar_Sector,Edgar_Subsector,Edgar_Subsector_Synthetic,Vars,ids,MEX,ssp_emission,missing_in_ssp_outputs
0,agrc,ch4,AG - Crops:CH4,Agriculture,AG - Crops,AG - Crops,emission_co2e_ch4_agrc_anaerobicdom_rice:emiss...,1:agrc:ch4,1.144799,0.595834,[]
1,agrc,co2,AG - Crops:CO2,Agriculture,AG - Crops,AG - Crops,emission_co2e_co2_agrc_biomass_bevs_and_spices...,2:agrc:co2,2.938056,0.793754,[]
2,agrc,n2o,AG - Crops:N2O,Agriculture,AG - Crops,AG - Crops,emission_co2e_n2o_agrc_biomass_burning:emissio...,3:agrc:n2o,29.452258,2.993998,[]
3,lvst,ch4,AG - Livestock:CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lvst_entferm_buffalo:emissio...,4:lvst:ch4,44.017738,126.073380,[]
4,lsmm,ch4,AG - Livestock:CH4,Agriculture,AG - Livestock,AG - Livestock,emission_co2e_ch4_lsmm_anaerobic_digester:emis...,5:lsmm:ch4,44.017738,7.325027,[]


### Create diff report

In [9]:
# subset the emission targets to create the diff report template
diff_report_df = emission_targets_df_extended[[
    "Subsector",
    "Edgar_Class",
    ISO3,
    "ssp_emission",
]].copy()
diff_report_df.head()

,Subsector,Edgar_Class,MEX,ssp_emission
0,agrc,AG - Crops:CH4,1.144799,0.595834
1,agrc,AG - Crops:CO2,2.938056,0.793754
2,agrc,AG - Crops:N2O,29.452258,2.993998
3,lvst,AG - Livestock:CH4,44.017738,126.073380
4,lsmm,AG - Livestock:CH4,44.017738,7.325027


In [10]:
# merge subsector an id into a single column for clarity
diff_report_df["subsector_id"] = diff_report_df["Subsector"] + " - " + diff_report_df["Edgar_Class"]
diff_report_df = diff_report_df.drop(columns=["Subsector", "Edgar_Class"])
diff_report_df.head()

,MEX,ssp_emission,subsector_id
0,1.144799,0.595834,agrc - AG - Crops:CH4
1,2.938056,0.793754,agrc - AG - Crops:CO2
2,29.452258,2.993998,agrc - AG - Crops:N2O
3,44.017738,126.073380,lvst - AG - Livestock:CH4
4,44.017738,7.325027,lsmm - AG - Livestock:CH4


In [11]:
#rename region column
diff_report_df = diff_report_df.rename(columns={ISO3: "inventory_emission"})
diff_report_df.head()

,inventory_emission,ssp_emission,subsector_id
0,1.144799,0.595834,agrc - AG - Crops:CH4
1,2.938056,0.793754,agrc - AG - Crops:CO2
2,29.452258,2.993998,agrc - AG - Crops:N2O
3,44.017738,126.073380,lvst - AG - Livestock:CH4
4,44.017738,7.325027,lsmm - AG - Livestock:CH4


In [12]:
# Create inventory_share column
diff_report_df["inventory_share"] = diff_report_df["inventory_emission"] / diff_report_df["inventory_emission"].sum()
diff_report_df

,inventory_emission,ssp_emission,subsector_id,inventory_share
0,1.144799,0.595834,agrc - AG - Crops:CH4,0.001792
1,2.938056,0.793754,agrc - AG - Crops:CO2,0.004600
2,29.452258,2.993998,agrc - AG - Crops:N2O,0.046112
3,44.017738,126.073380,lvst - AG - Livestock:CH4,0.068916
4,44.017738,7.325027,lsmm - AG - Livestock:CH4,0.068916
...,...,...,...,...
69,60.038853,31.663506,waso - Waste - Solid Waste:CH4,0.094000
70,0.499233,61.326295,waso - Waste - Solid Waste:CO2,0.000782
71,0.029932,0.061465,waso - Waste - Solid Waste:N2O,0.000047
72,23.362827,16.475465,trww - Waste - Wastewater Treatment:CH4,0.036578


In [13]:
# Calculate error column, avoid division by zero by adding a small constant to the denominator
epsilon = 1e-8
diff_report_df["error"] = (diff_report_df["ssp_emission"] - diff_report_df["inventory_emission"]).abs() / (diff_report_df["inventory_emission"] + epsilon)
diff_report_df["squared_error"] = (diff_report_df["error"] ** 2)
diff_report_df.head()

,inventory_emission,ssp_emission,subsector_id,inventory_share,error,squared_error
0,1.144799,0.595834,agrc - AG - Crops:CH4,0.001792,0.479529,0.229948
1,2.938056,0.793754,agrc - AG - Crops:CO2,0.004600,0.729837,0.532662
2,29.452258,2.993998,agrc - AG - Crops:N2O,0.046112,0.898344,0.807022
3,44.017738,126.073380,lvst - AG - Livestock:CH4,0.068916,1.864149,3.475053
4,44.017738,7.325027,lsmm - AG - Livestock:CH4,0.068916,0.833589,0.694871


In [14]:
# Set subsector_id at the beginning of the df
diff_report_df = diff_report_df[[
    "subsector_id",
    "inventory_emission",
    "ssp_emission",
    "inventory_share",
    "error",
    "squared_error"
]]

# sort by squared_error descending
diff_report_df = diff_report_df.sort_values(by="squared_error", ascending=False)
diff_report_df.head(10)

,subsector_id,inventory_emission,ssp_emission,inventory_share,error,squared_error
64,frst - LULUCF - HWP:CO2,0.000000,-42.853133,0.000000,4.285313e+09,1.836391e+19
66,soil - LULUCF - Organic Soil:N2O,0.000000,37.210194,0.000000,3.721019e+09,1.384599e+19
62,frst - LULUCF - Forest Land Removals:CO2,0.000000,36.040187,0.000000,3.604019e+09,1.298895e+19
65,soil - LULUCF - Organic Soil:CO2,0.000000,7.450540,0.000000,7.450540e+08,5.551054e+17
61,frst - LULUCF - Forest Land:CH4,0.000000,1.880314,0.000000,1.880314e+08,3.535582e+16
59,lndu - LULUCF - Deforestation:CH4,0.000000,0.855117,0.000000,8.551173e+07,7.312255e+15
57,ippu - IN - Industrial Processes:PFC,0.000000,0.000009,0.000000,8.906997e+02,7.933460e+05
70,waso - Waste - Solid Waste:CO2,0.499233,61.326295,0.000782,1.218411e+02,1.484526e+04
54,ippu - IN - Industrial Processes:HFC,0.751951,22.724061,0.001177,2.922014e+01,8.538167e+02
67,lndu - LULUCF - Other Land:CO2,-2.128000,-61.829050,-0.003332,-2.805500e+01,7.870833e+02


In [15]:
diff_report_df.tail(40)

,subsector_id,inventory_emission,ssp_emission,inventory_share,error,squared_error
37,ippu - IN - Industrial Processes:HFC,0.751951,0.000000e+00,0.001177,1.000000,1.000000
30,ippu - IN - Industrial Processes:HFC,0.751951,0.000000e+00,0.001177,1.000000,1.000000
33,ippu - IN - Industrial Processes:HFC,0.751951,0.000000e+00,0.001177,1.000000,1.000000
27,ippu - IN - Industrial Processes:HFC,0.751951,0.000000e+00,0.001177,1.000000,1.000000
31,ippu - IN - Industrial Processes:HFC,0.751951,0.000000e+00,0.001177,1.000000,1.000000
35,ippu - IN - Industrial Processes:HFC,0.751951,1.520355e-09,0.001177,1.000000,1.000000
38,ippu - IN - Industrial Processes:HFC,0.751951,8.905477e-06,0.001177,0.999988,0.999976
56,ippu - IN - Industrial Processes:HFC,0.751951,1.056646e-04,0.001177,0.999859,0.999719
39,ippu - IN - Industrial Processes:HFC,0.751951,1.730338e-02,0.001177,0.976989,0.954507
58,ippu - IN - Industrial Processes:SF6,0.544237,1.754959e-02,0.000852,0.967754,0.936547


### Save diff table

In [16]:
diff_report_df.to_csv(os.path.join(RUN_DIR_PATH, f"diff_report_{REGION_NAME}.csv"), index=False)